In [178]:
####################################
#  Analysing NGS coverage metrics
#  Amin Boroomand
#  Jan 2023, Verson 1
#  
#
###################################

I decompressed files in bash using the following commands


gzip -d *.gz    
gzip -k *.txt 

In [179]:
######################Imports########################

import pandas as pd
import os

In [180]:
#####################Reading files#####################

#set working directory
os.chdir("/Users/amin/Desktop/Cancer_Maftool/IGM_user01/")
# list of file names and dataframe names
files_list = ['sample1_depths.txt', 'sample2_depths.txt', 'sample3_depths.txt']
#put each file in a separated dataframe for each sample
for filename in files_list:
    if int(filename[6:7])==1:
        S1=pd.read_csv(filename, delimiter='\t')
    elif int(filename[6:7])==2:
        S2=pd.read_csv(filename, delimiter='\t')
    elif int(filename[6:7])==3:
        S3=pd.read_csv(filename, delimiter='\t')
        

In [181]:
#########Data_cleaning######################

#remove entries with 0 read depth from dataframes
S1 = S1.loc[S1['Depth'] != 0]
S2 = S2.loc[S2['Depth'] != 0]
S3 = S3.loc[S3['Depth'] != 0]
#We dont need the Depth column to answer question 1 and 2, so lets remove it
S1d = S1.drop('Depth', axis=1)
S2d = S2.drop('Depth', axis=1)
S3d = S3.drop('Depth', axis=1)


In [190]:
#############finding unique rows of sample 2 in comparison to sample 3 and Sample 1########

#concat sample 1 and sample 3
S1d_S3d = pd.concat([S1d, S3d], ignore_index=True)
#remove the dublicated rows of sample 1 and sample 3
S1d_S3d.drop_duplicates(inplace=True)

# Merge sample 2 with concatenated Sample 1 and sample3, then add an indicator column
merged_S2_concat = pd.merge( S2d, S1d_S3d, on=["Chromosome", "Position"], how='outer', indicator=True)
# Get the rows that are only present in sample 2
unique_rows_S2d = merged_S2_concat.loc[merged_S2_concat._merge == 'left_only']
# Drop the indicator column
unique_rows_S2d = unique_rows_S2d.drop(columns=['_merge'])
#print(unique_rows_S2d)
print("Question 1")
print("Number of positions that are unique to the file 'sample2_depths.txt' is = ", unique_rows_S2d.shape[0])


        Chromosome  Position
0             chr1      9997
692           chr1     10700
693           chr1     10701
694           chr1     10702
695           chr1     10703
...            ...       ...
2407483       chr3    982698
2407484       chr3    982699
2407485       chr3    982700
2407486       chr3    982701
2407487       chr3    982785

[25997 rows x 2 columns]
Question 1
Number of positions that are unique to the file 'sample2_depths.txt' is =  25997


In [191]:
#####################Number of positions that are in common between all three depth files##################

#merge the three DataFrames while only keep the rows that have matching values in all three of them
merged_df = pd.merge(pd.merge(S1d, S2d, on=["Chromosome", "Position"], how='inner'), S3d, on=["Chromosome", "Position"], how='inner')
print("Question 2")
print("Number of positions that are in common between all three depth files = ", merged_df.shape[0])

Question 2
Number of positions that are in common between all three depth files =  2354995


In [192]:
##########Computing the Average Depth of three samples###########

#concat all samples and calculate the sum of the Depth
depth_sum_df= pd.concat([S1, S2, S3]).groupby(['Chromosome', 'Position']).sum().reset_index()
#Add a column of Average of the Depth
depth_avg_df= depth_sum_df.assign(AVG_Depth=depth_sum_df['Depth'] / 3)

#Question 3: the average read depth for each chromosome 
Chromosome_AVG_Depth = depth_avg_df.groupby('Chromosome')['AVG_Depth'].sum()/1000000
print("Question 3")
print("Following is the average read depth for each chromosome in the experiment")
print(Chromosome_AVG_Depth)

Question 3
Following is the average read depth for each chromosome in the experiment
Chromosome
chr1    32.168590
chr2    66.668454
chr3    30.070197
Name: AVG_Depth, dtype: float64


In [193]:
#Question 4: find the row with a maximum average Depth
top_n_rows = depth_avg_df.nlargest(1, 'AVG_Depth')
print("Question 4")
print("Following is the position which has the largest average depth")
print(top_n_rows.drop(columns=['Depth']).to_string(index=False))

Question 4
Following is the position which has the largest average depth
Chromosome  Position    AVG_Depth
      chr1    567579  1660.666667
